# BERT Model Exploration

This notebook illustrates the setup of a BERT model used for determining similar responses to a patients query based on the the data it has previously seen.

In [ ]:
# gather all imports

import pandas as pd
import numpy as np
import json
from datasets import Dataset
import torch
from transformers import AutoTokenizer
from transformers import  AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import json

C:\Users\henry\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We will start by loading in all the data previously processed.

In [2]:
def load_and_prepare(file_path):
    """
    Helper function to load in the data into a specific form 

    @PARAMS:
        - file_path -> the file to process
    """
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        
        # format data to be just question answer pairs
        formatted_data = []
        for entry in data:
            formatted_data.append({
                "question": entry["Question"],
                "response": entry["Answer"]
            })
        
        print(f"Loaded {len(formatted_data)} Q&A pairs from {file_path}!")
        return formatted_data
        
    except Exception as e:
        print(f"Error loading in file...\n{e}")
        return []

In [3]:
# load in formatted data
## TRAIN ##
train_data = load_and_prepare("processed_data/train.json")

## VAL ##
val_data = load_and_prepare("processed_data/validation.json")

## TEST ##
test_data = load_and_prepare("processed_data/test.json")

# print out one value of each to make sure it is loaded correctly
print(train_data[0])
print(val_data[0])
print(test_data[0])

Loaded 18749 Q&A pairs from processed_data/train.json!
Loaded 2344 Q&A pairs from processed_data/validation.json!
Loaded 2344 Q&A pairs from processed_data/test.json!
{'question': 'will eating late evening meals increase my cholesterol?', 'response': 'no. it is what you are eating (as well as your genetics) not when you eat it. it depends on the kinds of foods that you eat. make sure that you are eating healthy foods in order to not gain great amount of cholesterol. you have to always watch what you eat in order to have a healthy skin and body. you may check out www. clearclinic. com for great ideas to achieve an acne free skin.'}
{'question': 'who is affected by arthritis?', 'response': 'arthritis sufferers include men and women children and adults. approximately 350 million people worldwide have arthritis. nearly 40 million people in the united states are affected by arthritis including over a quarter million children! more than 27 million americans have osteoarthritis. approximately

## BERT

For the model, I will use a BERT pre-trained model to build a set of recomendations based on a confidence score. Meaning that I will not fine-tune the model based on the patient doctor interactions. The goal is to build a text generation model and pass in the recomendations from here as an input. This is a hybrid approach.

In [ ]:
class BERT:
    """
    This class is the implementation of a BERT recommendation model.
    Without tuning, this model will recomend a response based on a set of confidence values that come from the set of all interactions.

    This will be passed to a generative model.
    """

    def __init__(self, model_name="bert-base-uncased", use_gpu=True):
        """
        Initializer function to establish the pre-trained model

        @PARAMS:
            - model_name -> specifc pre-trained model to initialize
            - use_gpu    -> speed up runtime by using GPU if available
        """
        # empty cache if possible
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        self.device = torch.device("cuda" if torch.cuda.is_available() and use_gpu else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(self.device)

        # values for the response
        self.data = None
        self.embeddings = None

    def get_embeddings(self, questions, batch_size=16):
        """
        Function to return a set of question embeddings.
        
        @PARAMS:
            - questions  -> the patient query
            - batch_size -> batch split to save time
        """
        embeddings = []

        with torch.no_grad():
            # process in batches to save time
            for i in range(0, len(questions), batch_size):
                batch_questions = questions[i:i + batch_size]
                
                # tokenize this batch
                inputs = self.tokenizer(
                    batch_questions,
                    padding=True,
                    truncation=True,
                    max_length=512,
                    return_tensors="pt"
                ).to(self.device)
                
                # get the BERT outputs
                outputs = self.model(**inputs)
                
                # convert to numpy
                batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
                embeddings.extend(batch_embeddings)
                
                # print progress!!
                if (i + 1) % (batch_size * 10) == 0:
                    print(f"Processed {i + 1}/{len(questions)} questions")
                
        return np.array(embeddings)
    
    def embed_data(self, data, batch_size=16):
        """
        Function to convert the data into a form that includes the embeddings.
        
        @PARAMS:
            - data       -> patient and doctor interactions
            - batch_size -> batch split to save time
        """
        self.data = data

        questions = [point['question'] for point in data]
        #responses = [point['response'] for point in data]

        # now get the embeddings for the q/r
        embeddings = self.get_embeddings(questions, batch_size)
        self.embeddings = embeddings

        # finally return all info with the embeddings
        return Dataset.from_dict({
            'question': questions,
            'embedding': embeddings.tolist()
        })
    
    def get_similar_responses(self, question, amount=3, threshold=.7):
        """ 
        Function to search through all the data and get the similar interaction cases.

        @PARAMS:
            - question  -> the input to search against
            - amount    -> how many to return
            - threshold -> cosine similarity results must return above this
        """
        if self.data is None or self.embeddings is None:
            raise ValueError("No data accessible...")
        
        # start by getting the embedding for the question input
        question_embedding = self.get_embeddings([question])

        # now run cosine similarity against the previously calculated data embeddings
        results = cosine_similarity(question_embedding, self.embeddings)[0]
        # based on the amount input, get those top values
        top_indices = np.argsort(results)[-amount:][::-1]

        # now loop through the top values and append only the ones that meet the threshold
        similar = []
        for index in top_indices:
            if results[index] >= threshold:
                similar.append({
                    'question': self.data[index]['question'],
                    'response': self.data[index]['response'],
                    'similarity_score': float(results[index]),
                })
        return similar
    
    def get_recommendation(self, question):
        """ 
        Function to return a recommendation of a response based on a user input.

        @PARAMS:
            - question -> the input from the patient waiting for the returned doctor response
        """
        # first get all similar cases
        similar_cases = self.get_similar_responses(question)

        if similar_cases:
            return {
                "similarity_score": similar_cases[0]['similarity_score'],
                "most_similar_question": similar_cases[0]['question'],
                "recommendation": similar_cases[0]['response'],
                "other_matches": [
                    {
                        'similarity_score': case['similarity_score'], 
                        'question': case['question'],
                        "response": case['response']
                    } for case in similar_cases[1:]
                ]
            }
        # o/w return a default answer suggesting for seeking other advice
        else:
            return {
                "similarity_score": 0.0,
                "recommendation": "I recommend consulting a healthcare provider for a proper evaluation.",
                "other_matches": []
            }

Now lets run the model to see how it does against test inputs!

In [5]:
# build the baseline model!
baseline_bert = BERT()
baseline_bert.embed_data(train_data)

# now loop through all the testing examples and write to a file the specific outputs!    
with open("BERT_BASELINE_results.txt", 'w', encoding='utf-8') as f:
    f.write("BERT MODEL EVALUATION RESULTS\n")
    f.write("=" * 80 + "\n\n")
    
    for i, test_case in enumerate(test_data, 1):
        f.write(f"Test Case {i}\n")
        f.write("-" * 80 + "\n\n")
        
        # Write the test question
        f.write("Patient Question:\n")
        f.write(test_case['question'])
        f.write("\n\n")
        
        # Write the actual doctor's response
        f.write("Actual Doctor's Response:\n")
        f.write(test_case['response'])
        f.write("\n\n")
        
        # Get model's recommendation
        result = baseline_bert.get_recommendation(test_case['question'])
        
        # Write model's response
        f.write("Model's Recommendation:\n")
        f.write(f"Similarity Score: {result['similarity_score']:.4f}\n")
        f.write(result['recommendation'])
        f.write("\n\n")
        
        # Write separator between test cases
        f.write("=" * 80 + "\n\n")

C:\Users\henry\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for an